In [ ]:
# The following codes are written by Pingjun

### Setup and load data set

In [ ]:
!pip install --no-cache-dir transformers sentencepiece
!pip install datasets
!pip install accelerate -U
!pip install wandb
import wandb
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, TrainerCallback
from sklearn.metrics import precision_score, recall_score, accuracy_score, average_precision_score, f1_score
import torch
import numpy as np
import pickle

In [ ]:
annotations = datasets.load_dataset("bigbio/sem_eval_2024_task_2", name="sem_eval_2024_task_2_source")

In [ ]:
raw_texts = datasets.load_dataset("bigbio/sem_eval_2024_task_2", name="sem_eval_2024_task_2_ct")["train"]

In [ ]:
import pandas as pd

data = annotations['train']
df = pd.DataFrame(data)
data_dict = df.to_dict(orient='index')

In [ ]:
from collections import defaultdict
section_count = defaultdict(int)

for instance in data_dict.values():
  section_id = instance.get("section_id")
  section_count[section_id] += 1

for section_id, count in section_count.items():
  print(f"Section_id: {section_id}, Count: {count}")

Section_id: Intervention, Count: 396
Section_id: Eligibility, Count: 486
Section_id: Results, Count: 322
Section_id: Adverse Events, Count: 496


### Data instances selection

In [ ]:
"""

Distribution of the har2learn and ambiguous instances according to `Section_id`:

For each update round:

140 instances including 110 har2learn instances and 30 ambiguous instances. The corresponding Section_id distribution matches the percentages as above. 140 instances will be chosen randomly for each update round.
New statements and labels will be generated based on the given `type`, `CTRs`and `section_id`.

hard2learn:

- Section_id: Intervention, Count: 9
- Section_id: Eligibility, Count: 11
- Section_id: Results, Count: 6
- Section_id: Adverse Events, Count:29
- *otal Count: 55

ambiguous:

- Section_id: Intervention, Count: 3
- Section_id: Eligibility, Count: 9
- Section_id: Results, Count: 2
- Section_id: Adverse Events, Count: 1
- Total Count: 15

"""

In [ ]:
import random

# Batch of randomly selected hard2learn instances
counts = {"Intervention": 18,
          "Eligibility": 22,
          "Results": 12,
          "Adverse Events": 58}

filtered_data = {key: value for key, value in data_dict.items() if value["section_id"] in counts}

# Randomly select instances based on counts
selected_instances = {}
for instance_id, instance_data in filtered_data.items():
  section_id = instance_data["section_id"]
  if counts[section_id] > 0:
    selected_instances[instance_id] = instance_data
    counts[section_id] -= 1


In [ ]:
def format4excel(df):
  # Adding new columns for furthur steps

  # Adding new column "new_section_type" with random section type different from "section_id"
  section_types = list(set(df['section_id']))
  df['New section type'] = df.apply(lambda x: random.choice([s for s in set(df['section_id'].unique()) if s != x['section_id']]), axis=1)

  # Adding new columns "New statement" with empty values
  df["New statement"] = ""

  # Adding new column "New Label" with a random label of "Contradiction" or "Entailment"
  df["New label"] = df["label"].apply(lambda x:random.choice(["Contradiction", "Entailment"]))

  # Adding new coloum "Problem Category"
  df["Problem Category"] = ""
  return df

In [ ]:
df_selected = pd.DataFrame(selected_instances).T
df_selected_formated = format4excel(df_selected)

file_name = "selected_instances_hard2learn.xlsx"
df_selected_formated.to_excel(file_name, index_label="id")


In [ ]:
# Batch of randomly selected ambiguous instances

counts = {"Intervention": 6,
          "Eligibility": 18,
          "Results": 4,
          "Adverse Events": 2}

filtered_data = {key: value for key, value in data_dict.items() if value["section_id"] in counts}

# Randomly select instances based on counts
selected_instances = {}
for instance_id, instance_data in filtered_data.items():
  section_id = instance_data["section_id"]
  if counts[section_id] > 0:
    selected_instances[instance_id] = instance_data
    counts[section_id] -= 1


In [ ]:
df_selected = pd.DataFrame(selected_instances).T
df_selected_formated = format4excel(df_selected)

file_name = "selected_instances_ambiguous.xlsx"
df_selected_formated.to_excel(file_name, index_label="id")

In [ ]:
# Save files to Drive
from google.colab import drive
import shutil
drive.mount('/content/drive')

In [ ]:
source_file_path_0 = '/content/selected_instances_hard2learn.xlsx'
destination_file_path_0 = '/content/drive/My Drive/selected_instances_hard2learn.xlsx'

shutil.copyfile(source_file_path_0, destination_file_path_0)

In [ ]:
source_file_path_1 = '/content/selected_instances_ambiguous.xlsx'
destination_file_path_1 = '/content/drive/My Drive/selected_instances_ambiguous.xlsx'

shutil.copyfile(source_file_path_1, destination_file_path_1)

### Read from file and assign problem categories according to distribution from evidence retrival

In [ ]:
!pip install pandas openpyxl google-auth google-auth-oauthlib google-auth-httplib2

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
file_path_0 = '/content/drive/My Drive/selected_instances_hard2learn.xlsx'
df_hard2learn = pd.read_excel(file_path_0)
file_path_1 = '/content/drive/My Drive/selected_instances_ambiguous.xlsx'
df_ambiguous = pd.read_excel(file_path_1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
merged_df = pd.concat([df_hard2learn, df_ambiguous])

In [ ]:
subcategories = ["Subcategory-category", "Numerical", "Synonym", "Eligibility based", "Intervention based", "Results based", "Abbreviation"]
distribution = [0.15, 0.45, 0.09, 0.13, 0.08, 0.03, 0.07] # Adjust the distribution as needed

In [ ]:
# Randomly assign subcategories
merged_df['Problem Category'] = np.random.choice(subcategories, size=len(merged_df), p=distribution)

In [ ]:
import shutil
source_path = "/content/selected_instances_merged.xlsx"
destination_path = "/content/drive/MyDrive/"  # Change this to your desired Google Drive folder

shutil.copy(source_path, destination_path)